In [1]:
import uproot

In [8]:
import torch
from torch_geometric.nn import EdgeConv, GCNConv
from torch_geometric.loader import DataLoader

In [9]:
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from graph_utils import load_tree


class HGCALTracksters(InMemoryDataset):
    
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['tracksters_ds_10e.root']

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):

        filename = f"{self.root}/{self.raw_file_names[0]}"
        tracksters = uproot.open({filename: "tracksters"})

        dataset = []

        for g, label in load_tree(tracksters):
            x = torch.tensor([pos for _, pos in g.nodes("pos")])
            edge_index = torch.tensor(list(g.edges())).T
            y = torch.tensor(label)
            dataset.append(Data(x, edge_index=edge_index, y=y))

        data, slices = self.collate(dataset)
        torch.save((data, slices), self.processed_paths[0])

In [10]:
ds = HGCALTracksters("data")
loader = DataLoader(ds, batch_size=32, shuffle=True)
ds

Processing...
Done!


HGCALTracksters(2441)

In [ ]:
print(f"PyTorch version: {torch.__version__}")
# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
# print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
# print(f"Is MPS available? {torch.backends.mps.is_available()}")
# device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu")

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

PyTorch version: 1.13.0.dev20220622
Using device: cpu


In [ ]:
class TracksterClassifier(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)
        # TODO:
        # graph pooling
        # dense layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

In [92]:
# prepare data
train_set = ds[:-1000]
test_set = ds[-1000:]
test_set[0]

Data(x=[8, 3], edge_index=[2, 22], y=[1])

In [93]:
model = TracksterClassifier(ds.num_node_features, 16)

In [94]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [95]:
def train(loader):
    for batch in loader:
        model.train()
        batch = batch.to(device)
        optimizer.zero_grad()
        z = model.encode(batch.x, batch.edge_index)
        loss = model.recon_loss(z, batch.pos_edge_label_index)
        loss.backward()
        optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

In [96]:
for epoch in range(1, 10):
    loss = train(train_set)
    auc, ap = test(test_set)
    print(f'Epoch: {epoch:03d}, AUC: {auc:.4f}, AP: {ap:.4f}')

AttributeError: 'TracksterClassifier' object has no attribute 'encode'